# [Bonus problem from Infi](https://aoc.infi.nl/)

In [1]:
{-# LANGUAGE NegativeLiterals #-}
{-# LANGUAGE FlexibleContexts #-}
{-# LANGUAGE OverloadedStrings #-}
{-# LANGUAGE TypeFamilies #-}

In [95]:
-- import Prelude hiding ((++))
import Data.Text (Text)
import qualified Data.Text as T
import qualified Data.Text.IO as TIO

import Text.Megaparsec hiding (State)
import qualified Text.Megaparsec.Lexer as L
import Text.Megaparsec.Text (Parser)
import qualified Control.Applicative as CA

import Data.List (nub, sort)

In [3]:
type Position = (Integer, Integer)

In [5]:
sc :: Parser ()
sc = L.space (skipSome spaceChar) CA.empty CA.empty

lexeme = L.lexeme sc
integer = lexeme L.integer
signedInteger = L.signed sc integer
symbol = L.symbol sc
comma = symbol ","

pointP :: Parser Position
pointP = (,) <$> signedInteger <* comma <*> signedInteger

startPosP = between (symbol "[") (symbol "]") pointP
stepP = between (symbol "(") (symbol ")") pointP

descriptionP = (,) <$> (some startPosP) <*> (some stepP)
-- descriptionP = (,) <$> (startPosP `sepBy` space) <*> (stepP `sepBy` space)

In [6]:
parseTest (some stepP) "(1,2)(3,4)"

[(1,2),(3,4)]

In [7]:
successfulParse :: Text -> ([Position], [Position])
successfulParse input = 
        case parse descriptionP "input" input of
                Left  _error -> ([], [])
                Right description -> description

In [8]:
sampleT = T.pack "[0,0][1,1](1,0)(0,-1)(0,1)(-1,0)(-1,0)(0,1)(0,-1)(1,0)"
sample = successfulParse sampleT
sample

([(0,0),(1,1)],[(1,0),(0,-1),(0,1),(-1,0),(-1,0),(0,1),(0,-1),(1,0)])

In [9]:
chunks :: Int -> [b] -> [[b]]
chunks n xs = (take n xs) : if null xs' then [] else chunks n xs'
    where xs' = drop n xs

In [10]:
chunks 3 "abcdefghijkl"

["abc","def","ghi","jkl"]

In [11]:
(starts, unchunkedSteps) = successfulParse sampleT
steps = chunks (length starts) unchunkedSteps
(starts, steps)

([(0,0),(1,1)],[[(1,0),(0,-1)],[(0,1),(-1,0)],[(-1,0),(0,1)],[(0,-1),(1,0)]])

In [15]:
(+:) (a, b) (c, d) = (a + c, b + d)

In [23]:
-- applySteps = zipWith (+:)

In [22]:
zipWith (+:) starts (head steps)

[(1,0),(1,0)]

In [24]:
scanl applySteps starts steps

[[(0,0),(1,1)],[(1,0),(1,0)],[(1,1),(0,0)],[(0,1),(0,1)],[(0,0),(1,1)]]

In [32]:
length $ filter ((1 ==) . length . nub) $ scanl applySteps starts steps

2

In [63]:
visited = scanl (zipWith (+:))

In [64]:
intersections = filter ((== 1) . length . nub)

In [65]:
part1 = length . intersections

In [78]:
bounds ps = ( minimum $ map fst ps
            , maximum $ map fst ps
            , minimum $ map snd ps
            , maximum $ map snd ps
            )

In [118]:
showPoints (minr, maxr, minc, maxc) ps = unlines [ [ if (r, c) `elem` ps then '*' else ' ' | r <- [minr..maxr] ] | c <- [minc..maxc] ]

In [127]:
main :: IO ()
main = do 
        text <- TIO.readFile "../../data/infi.txt"
        let (starts, unchunkedSteps) = successfulParse text
        let steps = chunks (length starts) unchunkedSteps
        let points = visited starts steps
        print $ part1 points
        let bds = bounds $ nub $ concat points
        putStrLn $ showPoints bds $ nub $ concat $ intersections points

In [128]:
main

535
                                   *****   *** 
                              **********  *****
                             ***********  *****
                             **********   *****
                             *****         *** 
                            *****              
                            ****               
  **        ********        ****               
****     **************     *******************
****    ****************    *******************
****   ******************   *******************
****   *****        *****   *****         *****
****   ****          ****   ****           ****
****   ****          ****   ****           ****
****   ****          ****   ****           ****
****   ****          ****   ****           ****
****   ****          ****   ****           ****
****   ****          ****   ****           ****
****   ****          ****   ****           ****
****   ****          ****   ****           ****
****   ****          ****   ****    

In [107]:
showPoints $ nub $ concat $ visited starts steps

"  \n  \n"

In [114]:
starts

[(0,0),(1,1)]